# Extraction, preprocessing and saving dataset

Fron csv data, we extract, process and save the dataset


In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import visuals as vs

In [33]:
#reading the csv (tsv) and returning selected data
def extract_feature_vectors(path):
    
    cols = ['RecordingDate','[Age]Value','[AQ Range]Value','[Category]Value','[Gender]Value','RecordingDuration','RecordingResolution'
    ,'RecordingTimestamp','EyeTrackerTimestamp','GazePointIndex','GazePointX (ADCSpx)','GazePointY (ADCSpx)'
        ,'StudioEvent','StudioEventIndex','StrictAverageGazePointX (ADCSmm)','StrictAverageGazePointY (ADCSmm)' ]
    data = pd.read_csv (path, sep = '\t', usecols = cols)
    features = data['RecordingTimestamp'] #T in ms. Our time Base
    eventMarks = data['StudioEvent']
    trackerTime = data['RecordingTimestamp']
    #extracting positions with nan values due to tracking errors
    
    GazeX = np.array(data['GazePointX (ADCSpx)'])
    GazeY = np.array(data['GazePointY (ADCSpx)'])    

    events = data['StudioEventIndex']
    return eventMarks,trackerTime,GazeX,GazeY


In [34]:
def calculate_velocities_from_position( time, positionArray):
    velocity = []
    timeRescaled = []
    time = [item for item in time]
    for x in range(1,len(time)):
        velocity.append((positionArray[x-1]-positionArray[x])/(time[x-1]-time[x]))
        timeRescaled.append(x)
    return np.array(velocity), np.array(timeRescaled)

In [39]:
def save_vector_to_file(fileName, col1,col2):
    path = 'register/'
    outFile = open(path + fileName,'w')
    outFile.write("time \t value\n")
    for x in range(len(col1)):
        outFile.write("{0} {1}\n".format(col1[x],col2[x]))
       # outFile.write('%10.10f %10.10f' %col1[x] %col2[x])
        
    outFile.close()
    

In [217]:
from scipy import signal as s
import math 
def show_spectrogram(time, signal):
    np.random.seed(52346234)
    nfft = 50   
    fs = 1.0/0.003332
    fig, (ax1, ax2) = plt.subplots(ncols = 2)

    ax1.plot(time, signal)
    data,freqs,bins, im = ax2.specgram(signal, NFFT = nfft, Fs = fs, noverlap = nfft-10)
    ax2.pcolormesh(bins, freqs, 10 * np.log10(data))
    ax2.axis('tight')
    plt.show()


In [166]:
from scipy import signal as s
def save_spectrogram(time, signal, datasetRawPath, imageNumber):
    
    fig,ax = plt.subplots(1)
    fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
    ax.axis('off')
    rate = 1.0/0.003332
    pxx, freqs, bins, im = ax.specgram(signal, Fs=rate, noverlap=118, NFFT=128) 
    #pxx, freqs, bins, im = ax.specgram(signal, Fs=rate, noverlap=246, NFFT=256)# para el hd, pero se pierde calidad
    ax.axis('off')
    fig.savefig(datasetRawPath +'spec'+str(imageNumber)+".jpg", dpi=300, frameon='false')
    #plt.show()
    
   

In [1]:
def is_nan_free( vector):    
    nan_mask = np.isnan(np.min(vector))
    return not(nan_mask)
    


In [2]:
def save_visualpath_map( xVector,yVector,datasetPath, imageNumber):
    #para las velocidades xVector = time, y Vector = velocity
    plt.plot(xVector,yVector,'g-')
    plt.xlim(-100,1250)
    plt.ylim(-100,800)
    plt.axis('off')
    plt.savefig(datasetPath +'rp'+str(imageNumber)+".jpg", dpi=300, frameon='false')
    plt.show()

In [162]:
"""segmentation of the temporal gaze positiondata in images and crosses segment, process
velocities, generate and save spectrograms as dataset.
"""
def extract_valid_features(eventMarks, trackerTime, GazeX, GazeY,numberIndex):
    startIndex = np.where(eventMarks == 'MovieStart')
    startIndex = np.array(startIndex)
    stopIndex = np.where(eventMarks == 'MovieEnd')
    trackerCountsPerSecond = 1000 #using RecordingTimestamp t in ms
   # startSegmentTime = trackerTime[(startIndex[0]+1)]
    startSegmentIndex=(startIndex[0][0])+1
    showDataFigures = 0
    base= numberIndex * 22
    for x in range (0,22):
        imageNumber= base+x
        print("SEGMENTO ",x)
        startSegmentTime = trackerTime[startSegmentIndex]
        targetT=(startSegmentTime+(2*1000))        
        endOfSegment = np.searchsorted(trackerTime, startSegmentTime+(2*1000))       
        crossSegmentTime = trackerTime[startSegmentIndex:(endOfSegment+1)]        
        posXCross = GazeX[startSegmentIndex:(endOfSegment+1)]
        posYCross = GazeY[startSegmentIndex:(endOfSegment+1)]
        
        if (is_nan_free(posXCross) and (is_nan_free(posYCross))):            
            
            velX, timeX = calculate_velocities_from_position(crossSegmentTime, posXCross)
            velY, timeY = calculate_velocities_from_position(crossSegmentTime, posYCross)
    
            if (showDataFigures == 1):
                vs.plot_position_xy("cruz "+str(x+1),posXCross,posYCross)
                vs.plot_2vectors("cross X",timeX,velX, posXCross[:len(posXCross)-1])
                vs.plot_2vectors("cross Y",timeY,velY,posYCross[:len(posYCross)-1])
                rec_plot(velX,"Cross vel X", True)
                rec_plot(velY,"Cross vel Y", True)
                show_spectrogram(timeY,velY)
                show_spectrogram(crossSegmentTime,posYCross)
                show_spectrogram(timeX,velX)
                show_spectrogram(crossSegmentTime,posXCross)
            save_spectrogram(timeX, velX,"dataset_raw_extended_hd/asd/cross/X/",imageNumber)
            save_spectrogram(timeY, velY,"dataset_raw_extended_hd/asd/cross/Y/",imageNumber)
            #save_spectrogram(timeX, velX,"dataset_raw_tania/td/cross/X/",imageNumber)
            #save_spectrogram(timeY, velY,"dataset_raw_tania/td/cross/Y/",imageNumber)
            #save_rec_plot_combined(velX,velY,"dataset_raw_rp_test/asd/cross/",imageNumber)
        else:
            print("\x1b[31m CROSS ",str(x)," DATA CORRUPTED""\x1b[0m")       
         
        startSegmentIndex = np.array(endOfSegment)+1
        startSegmentTime = trackerTime[startSegmentIndex]
        endOfSegment = np.searchsorted(trackerTime, startSegmentTime+(4*1000))
        targetT=(startSegmentTime+(4*1000))
               
        imgTimeSegment = trackerTime[startSegmentIndex:(endOfSegment+1)]
        posXImg = GazeX[startSegmentIndex:(endOfSegment+1)]
        posYImg = GazeY[startSegmentIndex:(endOfSegment+1)]
        #save_visualpath_map(posXImg,posYImg,"dataset_raw_visualpath_test/td/"+str(x+1)+"/",imageNumber )
        if ( is_nan_free(posXImg) and (is_nan_free(posYImg))):            
            
            velX, timeX = calculate_velocities_from_position(imgTimeSegment,posXImg)
            velY, timeY = calculate_velocities_from_position(imgTimeSegment,posYImg)
            
            if (showDataFigures == 1):
                vs.plot_vector("exploration"+str(x), posXImg, posYImg)
                vs.plot_2vectors("velX and posX", timeX,velX,posXImg )
                show_spectrogram(timeX,velX)
                show_spectrogram(imgTimeSegment,posXImg) 
                show_spectrogram(timeY,velY)
                show_spectrogram(imgTimeSegment,posYImg)
                rec_plot(velX,"Img vel X", True)
                rec_plot(velY,"Img vel Y", True)
            save_spectrogram(timeX, velX,"dataset_raw_extended_hd/asd/images/X/",imageNumber)
            save_spectrogram(timeY, velY,"dataset_raw_extended_hd/asd/images/Y/",imageNumber)
            #save_spectrogram(timeX,velX,"dataset_raw_extended_test/asd/images/X/",imageNumber)
            #save_spectrogram(timeY,velY,"dataset_raw_extended_test/asd/images/Y/",imageNumber)
            ##save_spectrogram(timeX,velX,"dataset_raw_tania/td/images/X/",imageNumber)
            #save_spectrogram(timeY,velY,"dataset_raw_tania/td/images/Y/",imageNumber)
            #save_rec_plot_combined(velX,velY,"dataset_raw_rp_test/asd/images/",imageNumber)
            
        else:
            print("\x1b[31m IMAGE ",str(x)," DATA CORRUPTED""\x1b[0m")
   
        
        startSegmentIndex = endOfSegment+1

# Read raw data folder and process content

In [ ]:
from glob import glob

# load filenames
files = np.array(glob("raw_data_classified/asd/*"))

print(len(files))
index= 0
for file_path in files:
    events,time,gazePosX,gazePosY = extract_feature_vectors(file_path)
    #print(file_path)
    extract_valid_features(events,time,gazePosX,gazePosY,index)
    index = index+1